In [2]:
import pandas as pd

In [4]:
faltas = pd.read_csv('../data/processed/analise_faltas.csv')

In [5]:
faltas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20953 entries, 0 to 20952
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   partida_id  20953 non-null  int64 
 1   data        20953 non-null  object
 2   vencedor    20953 non-null  object
 3   mandante    20953 non-null  object
 4   visitante   20953 non-null  object
 5   clube       20953 non-null  object
 6   cartao      20953 non-null  object
 7   atleta      20947 non-null  object
 8   posicao     19755 non-null  object
 9   minuto      20953 non-null  object
 10  resultado   20953 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.8+ MB


In [11]:
faltas.posicao.value_counts()

posicao
Meio-campo      8697
Zagueiro        7460
Atacante        2874
Desconhecido    1198
Goleiro          702
Zagueira          22
Name: count, dtype: int64

In [15]:
# agrupa faltas por cartao, posicao, tempo do jogo (faixas) e resultado
# usa variáveis já presentes no notebook (faltas, pandas importado)

# extrai minuto numérico (trata valores como '90+')
faltas['minuto_num'] = pd.to_numeric(faltas['minuto'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')

# cria faixas de tempo de jogo
bins = list(range(0,101))+[float('inf')]
#labels = ['0-15', '16-30', '31-45', '46-60', '61-75', '76-90', '90+']
faltas['tempo_jogo'] = pd.cut(faltas['minuto_num'], bins=bins, right=True, include_lowest=True)

# padroniza posicao nula
faltas['posicao'] = faltas['posicao'].fillna('Desconhecido')

# cria dataset agrupado
faltas_grouped = (
    faltas
    .groupby(['cartao', 'posicao', 'tempo_jogo', 'resultado', 'clube'])
    .size()
    .reset_index(name='ocorrencias')
    .sort_values(['ocorrencias'], ascending=False)
)

# exibe as primeiras linhas do agrupamento
faltas_grouped.head()

/tmp/ipykernel_28776/3252343433.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['cartao', 'posicao', 'tempo_jogo', 'resultado', 'clube'])


,cartao,posicao,tempo_jogo,resultado,clube,ocorrencias
40069,Amarelo,Meio-campo,"(89.0, 90.0]",Vitória,Flamengo,24
60659,Amarelo,Zagueiro,"(89.0, 90.0]",Vitória,Atletico-MG,22
60609,Amarelo,Zagueiro,"(89.0, 90.0]",Derrota,Gremio,22
40074,Amarelo,Meio-campo,"(89.0, 90.0]",Vitória,Internacional,21
60673,Amarelo,Zagueiro,"(89.0, 90.0]",Vitória,Flamengo,20


In [16]:
faltas_grouped.to_csv('../data/processed/analise_cartoes.csv', index=False)

In [17]:
faltas_grouped.posicao.value_counts()

posicao
Meio-campo      20604
Zagueiro        20604
Atacante        20604
Desconhecido    20604
Goleiro         20604
Zagueira        20604
Name: count, dtype: int64

In [12]:
faltas_grouped.tempo_jogo.value_counts()

tempo_jogo
(-0.001, 1.0]    36
(1.0, 2.0]       36
(2.0, 3.0]       36
(3.0, 4.0]       36
(4.0, 5.0]       36
                 ..
(96.0, 97.0]     36
(97.0, 98.0]     36
(98.0, 99.0]     36
(99.0, 100.0]    36
(100.0, inf]     36
Name: count, Length: 101, dtype: int64